<a href="https://colab.research.google.com/github/Kimeteor309/Financial-big-data-analysis/blob/main/%E9%87%91%E8%9E%8D%E5%A4%A7%E6%95%B8%E6%93%9A%E5%88%86%E6%9E%90_week10%E4%BD%9C%E6%A5%AD_%E9%87%91%E8%9E%8D%E6%96%87%E6%9C%AC%E6%83%85%E7%B7%92%E8%BE%A8%E8%AD%98Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 安裝所需的庫
!pip install transformers
!pip install datasets
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
# 引入所需的模組
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader
import torch

# 設置使用的設備（GPU 或 CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加載金融文本資料集
dataset = load_dataset("takala/financial_phrasebank", 'sentences_allagree')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.88k [00:00<?, ?B/s]

financial_phrasebank.py:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

The repository for takala/financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/takala/financial_phrasebank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


FinancialPhraseBank-v1.0.zip:   0%|          | 0.00/682k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

In [3]:
from sklearn.model_selection import train_test_split

# 加載BERT分詞器
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 轉換文本為BERT可以接受的格式
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

# 將資料集應用於分詞
encoded_dataset = dataset.map(tokenize_function, batched=True)

# 將資料集中完整的數據取出
full_dataset = encoded_dataset['train']

# 使用train_test_split將數據分為80%訓練集和20%測試集
train_size = 0.8
train_indices, test_indices = train_test_split(list(range(len(full_dataset))), test_size=1-train_size, random_state=42)

# 根據索引分割資料集
train_dataset = full_dataset.select(train_indices)
test_dataset = full_dataset.select(test_indices)

# 加載BERT分類模型，設置為三分類（positive, neutral, negative）
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
model.to(device)  # 將模型移到指定設備（如 GPU）

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/2264 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [4]:
# 定義訓練參數
training_args = TrainingArguments(
    output_dir="./results",          # 輸出結果的目錄
    num_train_epochs=3,              # 訓練的輪次
    per_device_train_batch_size=16,  # 每個設備的訓練批次大小
    per_device_eval_batch_size=16,   # 每個設備的評估批次大小
    warmup_steps=500,                # 預熱步驟
    weight_decay=0.01,               # 權重衰減
    logging_dir="./logs",            # 日誌的目錄
    logging_steps=10,
    evaluation_strategy="epoch",      # 每輪結束時進行評估
    report_to="none"  # 禁用 WandB 日誌
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
from transformers import DataCollatorWithPadding

# 初始化 collator 用於填充批次數據
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 訓練模型
trainer = Trainer(
    model=model,            # 模型
    args=training_args,        # 訓練參數
    train_dataset=train_dataset,   # 訓練資料集
    eval_dataset=test_dataset,    # 評估資料集
    data_collator=data_collator
)

# 開始訓練
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.679200,0.513463
2,0.336200,0.128078
3,0.116400,0.155411


TrainOutput(global_step=342, training_loss=0.44005172845923846, metrics={'train_runtime': 141.4106, 'train_samples_per_second': 38.42, 'train_steps_per_second': 2.418, 'total_flos': 357373799629056.0, 'train_loss': 0.44005172845923846, 'epoch': 3.0})

In [6]:
# 在測試集上進行評估
results = trainer.evaluate()

# 顯示評估結果
print(results)


{'eval_loss': 0.1554107666015625, 'eval_runtime': 3.132, 'eval_samples_per_second': 144.637, 'eval_steps_per_second': 9.259, 'epoch': 3.0}


In [7]:
# 進行預測
test_texts = [
  "The company's profit has increased significantly this quarter.",
  "The increase in costs negatively affected the revenue.",
  "The company's performance remained stable."
]

test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt").to(device)

# 推論
outputs = model(**test_encodings)

# 取得預測結果
preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()

# 將數字標籤轉換回文字標籤
label_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
predicted_labels = [label_map[pred] for pred in preds]
print(predicted_labels)


['positive', 'negative', 'positive']
